In [1]:
import matplotlib.pyplot as plt
import numpy as np
import plot_spheroid as ps

In [ ]:
#plot one spheroid with Teff variation across the surface
def flux_star(omega,n):

    
    def get_one_F(omega,theta):
        r,T,F=ps.solve_ELR(omega,theta)
        return F
    
    Re=1.0
    Rp=Rp_div_Re(omega)*Re
   
    # Set of all spherical angles:
    phi = np.linspace(-np.pi, np.pi, n)
    nu = np.linspace(-np.pi/2, np.pi/2, n)

    angles=np.outer(np.ones_like(phi),nu)
    F=np.outer(np.ones_like(phi),np.ones_like(nu))
    for j in range(n):
        theta=angles[0,j]
        F[:,j]=get_one_F(omega,theta)
        
    if(Rp==Re): #spherical
        a=Re
        x=a*np.outer(np.cos(phi),np.cos(nu))
        y=a*np.outer(np.sin(phi),np.cos(nu))
        z=a*np.outer(np.ones_like(phi), np.sin(nu))
        
        F=np.outer(np.ones_like(phi), np.ones_like(nu))
        
    else: #spheroidal
        mu=np.arctanh(Rp/Re)
        a=np.sqrt(Re*Re - Rp*Rp)

        # Cartesian coordinates that correspond to the spherical angles:
        # (this is the equation of an ellipsoid):
        x = a*np.cosh(mu) * np.outer(np.cos(phi), np.cos(nu))
        y = a*np.cosh(mu) * np.outer(np.sin(phi), np.cos(nu))
        z = a*np.sinh(mu) * np.outer(np.ones_like(phi), np.sin(nu))
        Fmin=F.min()
        Fmax=F.max()
        F=(F-Fmin)/(Fmax - Fmin)
        
        return x, y, z, F
def rotate_star(x, y, z, theta, u):
    
    R = np.array([[np.cos(theta) + u[0]**2 * (1-np.cos(theta)), 
             u[0] * u[1] * (1-np.cos(theta)) - u[2] * np.sin(theta), 
             u[0] * u[2] * (1 - np.cos(theta)) + u[1] * np.sin(theta)],
            [u[0] * u[1] * (1-np.cos(theta)) + u[2] * np.sin(theta),
             np.cos(theta) + u[1]**2 * (1-np.cos(theta)),
             u[1] * u[2] * (1 - np.cos(theta)) - u[0] * np.sin(theta)],
            [u[0] * u[2] * (1-np.cos(theta)) - u[1] * np.sin(theta),
             u[1] * u[2] * (1-np.cos(theta)) + u[0] * np.sin(theta),
             np.cos(theta) + u[2]**2 * (1-np.cos(theta))]])
    return 